In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
scripts_dataset = pd.read_csv('Simpsons_dataset/simpsons_script_lines.csv', error_bad_lines=False)

b'Skipping line 8084: expected 13 fields, saw 20\nSkipping line 52607: expected 13 fields, saw 21\nSkipping line 59910: expected 13 fields, saw 21\n'
b'Skipping line 71801: expected 13 fields, saw 20\nSkipping line 73539: expected 13 fields, saw 21\nSkipping line 77230: expected 13 fields, saw 21\nSkipping line 78953: expected 13 fields, saw 21\nSkipping line 81138: expected 13 fields, saw 20\nSkipping line 86746: expected 13 fields, saw 22\nSkipping line 101154: expected 13 fields, saw 21\nSkipping line 115438: expected 13 fields, saw 20\nSkipping line 117573: expected 13 fields, saw 22\nSkipping line 130610: expected 13 fields, saw 22\n'
b'Skipping line 152970: expected 13 fields, saw 22\nSkipping line 153017: expected 13 fields, saw 20\nSkipping line 153018: expected 13 fields, saw 30\nSkipping line 154080: expected 13 fields, saw 20\nSkipping line 154082: expected 13 fields, saw 20\nSkipping line 154084: expected 13 fields, saw 20\nSkipping line 154086: expected 13 fields, saw 20\n

In [3]:
scripts_dataset.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31.0
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3.0
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22.0
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5.0
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33.0


In [4]:
dates = pd.read_csv('Simpsons_dataset/simpsons_episodes.csv')

In [5]:
texts = scripts_dataset.dropna().groupby('episode_id')['spoken_words'].agg('sum')

In [6]:
dates = dates.set_index('id')

In [7]:
air_dates = dates.loc[texts.index, :]['original_air_date']

In [8]:
text_with_dates = pd.concat([air_dates, texts], axis=1)

In [9]:
from textblob import TextBlob


def get_text_sentiment(text):
    try:
        tb = TextBlob(text)
        sentences = np.array(tb.sentences)
        if sentences.dtype != np.object:
            return np.nan
        
        vfunc = np.vectorize(lambda x: x.sentiment.polarity)
        sentences = vfunc(sentences)
        return np.average(sentences)
    except Exception:
        return np.nan

In [ ]:
from tqdm import tqdm

tqdm.pandas()
sentiments = text_with_dates.loc[:, 'spoken_words'].progress_apply(get_text_sentiment)

 94%|█████████▍| 529/564 [00:50<00:02, 12.15it/s]

In [ ]:
text_with_dates['sentiment'] = sentiments
text_with_dates = text_with_dates.drop('spoken_words', axis=1)

In [18]:
plt.plot(text_with_dates.set_index('original_air_date'))
plt.show()

KeyError: "None of ['original_air_date'] are in the columns"